In [ ]:
from code import *
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


Création du dataset et chargement dans le DataLoader
la fonction `gen_csv(img_dir, path_csv)` génère le fichier csv des labels selon les dossier des images

In [4]:
CONFIG = {
    "label_file": "./labels/labels.csv",
    "img_dir": "./img",
    "optimizer":"SGD",
    "learning_rate":0.01,
    "epoch":50,
    "loss":"cross_entropy",
    "batch_size":16,
    "device":"cuda" if torch.cuda.is_available() else "cpu",
    "path_config":"./config.json", # PATH TO CHANGE
    "loss_history" : {"train":[],
                      "val": [],
                      },
    "overall_time": 0
}

In [ ]:
## Dataset
data = CustomImageDataset(CONFIG["label_file"], CONFIG["img_dir"])

## DataLoader
test_prop = 0.1
val_prop = 0.1
data, test_set = random_split(data, [1 - test_prop, test_prop])
train_set, val_set = random_split(data, [1 - val_prop, val_prop])
train_dataloader = DataLoader(train_set, batch_size=CONFIG["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=CONFIG["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=CONFIG["batch_size"], shuffle=True)
CONFIG["n_train"] = len(train_dataloader)
CONFIG["n_val"] = len(val_dataloader)

## Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

net = Net().to(CONFIG["device"])
print(net)